<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 07 NLP

## Задание 1

Переделать генерацию текста по символам в генерацию по словам

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd "./gdrive/My Drive/Colab Notebooks/NLP/"

/content/gdrive/My Drive/Colab Notebooks/NLP


In [4]:
path_to_file = 'evgenyi_onegin.txt'

# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


## Задание 2

Запустить seq2seq, seq2seq с внимаием для перевода русских слов + описать наблюдения по качеству